In [ ]:
from foundations.projectables import ProjectableModel
from modeling.compute import create_vars
from numpy import pi
import numpy as np
from foundations.coupledgrad import pure_functions, coupled_functions
from foundations.coupledgrad import solver, concat, compose, eliminate, couplingvars
np.set_printoptions(suppress=True)

## Hydro

In [2]:
Df, Ds, Dd, tf, ts, td, hf, mtot = create_vars('Df Ds Dd tf ts td hf m_{platform}')
g, rhow = 9.81, 1023.6

Hydro = ProjectableModel()
Vd = Hydro.Var('Vd', pi/4*(Df**2*hf+Ds**2*ts+Dd**2*td))
FB = Hydro.Var('F_B', rhow*Vd*g)
FW = Hydro.Var('F_W', FB)
# center of buoyancy above the keel calculations
xd,xs = td/2,td+ts/2
xf1,xf2 = td+ts+hf/2, td+ts+tf/2
totA1, totA2 = hf*Df+ts*Ds+td*Dd, tf*Df+ts*Ds+td*Dd
KB = Hydro.Var('K_B', (hf*Df*xf1+ts*Ds*xs+td*Dd*xd)/totA1) 
KG = Hydro.Var('K_G', (tf*Df*xf2+ts*Ds*xs+td*Dd*xd)/totA2) 
I = Hydro.Var('I', pi/64*Df**4)
BM = Hydro.Var('B_M', I/Vd)
#should be 3-5% according to http://web.mit.edu/13.012/www/handouts/Reading3.pdf
GM = Hydro.Var('G_M', KB+BM-KG) 
C33 = Hydro.Var('C_{33}', pi*rhow*g/4*Df**2)
A33 = Hydro.Var('A_{33}', 0.0525*pi*rhow*(Dd**3+Ds**3+Df**3))
omega0 = Hydro.Var('\omega_0', (C33/(A33+mtot))**1/2)

In [3]:
eqs = [(p.right, p.var) for p in Hydro.projectables]

In [4]:
fis =coupled_functions(eqs)
Hydro_data, Rdata = compose(fis)

In [5]:
x0 = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, td:0.1, hf:0.9*0.3, 
      mtot:800}
x0arr = [x0[vr] for vr in Hydro_data[2]]
np.array(Hydro_data[0](x0arr))

array([    1.47654855, 14826.78586136, 14826.78586136,     0.235     ,
           0.15      ,     0.78539816,     0.53191489,     0.61691489,
       31546.3528965 ,  4051.82514267,     3.25097793,     2.        ,
           2.        ,     2.        ,     0.27      ,     0.1       ,
           0.1       ,     0.1       ,   800.        ])

## Mass

In [6]:
mbatt, A_s = create_vars('m_{batt} A_s')
mprop, mcomms, eta_solar, rho, rhoh = 20, 20, 10, 700, 2700
Mass = ProjectableModel()
Mass.add_equation(mtot, FW/g)
msolar = Mass.Var('m_{solar}', eta_solar*A_s)
mstruct = Mass.Var('m_{struct}', mtot-mbatt-msolar-mcomms-mprop) 
Mass.add_equation(td, (4/pi*mstruct-Df**2*tf*rho-Ds**2*ts*rho)/(Dd**2*rhoh))

In [7]:
eqs = [(p.right, p.var) for p in Mass.projectables]
fis =coupled_functions(eqs)
Mass_data, _ = compose(fis)

# Coupled

In [8]:
eqs = [(p.right, p.var) for p in Hydro.projectables+Mass.projectables]
Fdata = coupled_functions(eqs)
Rdata = pure_functions([right-left for right, left in eqs])


In [9]:
feed_forward, residual_functions, solvevars = couplingvars(Fdata, Rdata)

In [10]:
Rreduced = solver(residual_functions, solvevars, [[1000,1]], 
                                             eliminate=feed_forward)

In [11]:
x0.update({mbatt:100, A_s:0.5})
x0arr = [x0[vr] for vr in Rreduced[2]]
zout = np.array(Rreduced[0](x0arr))

In [12]:
x0s={key:val for key,val in zip(solvevars,zout)}

In [23]:
y0 = {**x0, **x0s}
yH0 = [y0[vr] for vr in Hydro_data[2]]
zH = {key:val for key,val in zip(Hydro_data[1], np.array(Hydro_data[0](yH0)))}

In [27]:
z0 = {**x0, **zH}
yM0 = [z0[vr] for vr in Mass_data[2]]
zM = {key:val for key,val in zip(Mass_data[1], np.array(Mass_data[0](yM0)))}

In [28]:
zH

{Vd: 1.5232808932436375,
 F_B: 15296.049462000277,
 F_W: 15296.049462000277,
 K_B: 0.24243768387716275,
 K_G: 0.15743768387716273,
 I: 0.785398163397448,
 B_M: 0.5155964122447831,
 G_M: 0.6005964122447831,
 C_{33}: 31546.35289650436,
 A_{33}: 4051.825142670288,
 \omega_0: 2.811089027127928,
 Dd: 2.0,
 Df: 2.0,
 Ds: 2.0,
 hf: 0.27,
 td: 0.11487536775432543,
 ts: 0.1,
 tf: 0.1,
 m_{platform}: 1559.2303223241934}

In [29]:
zM

{m_{platform}: 1559.2303223241934,
 m_{solar}: 5.0,
 m_{struct}: 1414.2303223241934,
 td: 0.11487536775432543,
 F_W: 15296.049462000277,
 A_s: 0.5,
 m_{batt}: 100.0,
 Dd: 2.0,
 Df: 2.0,
 Ds: 2.0,
 tf: 0.1,
 ts: 0.1}